# Contract Clause Classification System using DSPy

This notebook implements an advanced contract clause classification system using DSPy and Azure OpenAI. The system employs chain-of-thought reasoning and zero-shot learning to identify specific clauses in legal contracts.

## Zero-Shot Learning with Chain-of-Thought
This implementation uses:
1. Chain-of-thought reasoning for detailed analysis
2. Zero-shot learning without requiring training examples
3. Legal-specific prompting with weighted metrics
4. K-fold cross-validation for robust evaluation

In [ ]:
import os 
import dspy
import pandas as pd
from dotenv import dotenv_values, load_dotenv
from openai import AzureOpenAI

# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
deployment = 'gpt-4o-mini-eastus-0718'

# Initialize DSPy with Azure OpenAI
turbo = dspy.AzureOpenAI(
    api_key=api_key,
    api_version="2024-06-01",
    api_base=azure_endpoint,
    model=deployment
)

# Configure DSPy
dspy.configure(lm=turbo)

In [ ]:
class ContractAnalyzer(dspy.Signature):
    context = dspy.InputField(desc="The contract text to analyze")
    analysis = dspy.OutputField(desc="Step-by-step analysis of contract structure")
    conclusion = dspy.OutputField(desc="Final summary of identified sections")

class ChainOfThoughtClassifier(dspy.Signature):
    context = dspy.InputField(desc="The contract text to classify")
    clause_type = dspy.InputField(desc="The type of clause to identify")
    reasoning = dspy.OutputField(desc="Step-by-step legal analysis of the clause presence")
    decision = dspy.OutputField(desc="Final classification (Present/Absent) with justification")

In [ ]:
class ContractPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyzer = dspy.Predict(ContractAnalyzer)
        self.classifier = dspy.Predict(ChainOfThoughtClassifier)
    
    def forward(self, contract_text, clause_type):
        # First, analyze contract structure
        analysis = self.analyzer(context=contract_text)
        
        # Then perform classification with reasoning
        result = self.classifier(
            context=contract_text,
            clause_type=clause_type
        )
        
        return result.decision, result.reasoning

In [ ]:
class ZeroShotOptimizer:
    def __init__(self, model):
        self.model = model
        self.legal_terms = ['pursuant to', 'hereinafter', 'clause', 'provision', 'agreement']
    
    def _evaluate_reasoning_quality(self, reasoning):
        # Check for legal terminology
        term_presence = sum(1 for term in self.legal_terms if term in reasoning.lower()) / len(self.legal_terms)
        
        # Check analysis depth (number of distinct points made)
        analysis_points = len(reasoning.split('.'))
        
        return 0.5 * term_presence + 0.5 * min(1.0, analysis_points / 5)
    
    def _evaluate_confidence(self, analysis):
        # Evaluate thoroughness of analysis
        sections = len(analysis.split('\n'))
        return min(1.0, sections / 3)
    
    def _evaluate_pipeline(self, pipeline, eval_set):
        total_score = 0
        for example in eval_set:
            pred = pipeline(example['context'], example['clause_type'])
            total_score += self.metric_fn(pred, example['gold'])
        return total_score / len(eval_set)
    
    def optimize(self, pipeline):
        # Create optimizer with multiple metrics
        optimizer = dspy.teleprompt.BasicOptimizer(
            metric="weighted_accuracy",
            num_rounds=5,
            max_iterations=10,
            temp_range=[0.0, 0.7],  # Temperature range to explore
            max_tokens_range=[100, 500]  # Response length range
        )
        
        # Enhanced metric function with semantic similarity
        def metric_fn(pred, gold):
            # Direct match score
            exact_match = pred.decision.lower() == gold.lower()
            
            # Reasoning quality score
            reasoning_score = self._evaluate_reasoning_quality(pred.reasoning)
            
            # Confidence score
            confidence_score = self._evaluate_confidence(pred.analysis)
            
            # Weighted combination of scores
            return 0.6 * exact_match + 0.2 * reasoning_score + 0.2 * confidence_score
        
        # Store metric_fn as instance attribute for _evaluate_pipeline
        self.metric_fn = metric_fn
        
        # Load and prepare validation data
        df = pd.read_csv('contracts_advanced/contract_labels.csv')
        df.columns = [col.lower() for col in df.columns]
        
        # Create comprehensive validation set
        valset = []
        for _, row in df.iterrows():
            with open(row['filename'], 'r') as f:
                contract_text = f.read()
                
            for clause, column in {
                "Non-Disclosure Agreement (NDA) clause": "contains_nda",
                "Termination Clause": "contains_termination",
                "Indemnity Clause": "contains_indemnity",
                "Force Majeure Clause": "contains_force_majeure",
                "Data Protection Clause": "contains_data_protection"
            }.items():
                if row[column] != "Unknown":
                    valset.append({
                        'context': contract_text,
                        'clause_type': clause,
                        'gold': row[column]
                    })
        
        # Implement k-fold cross-validation
        k = 5
        fold_size = len(valset) // k
        best_pipeline = None
        best_score = 0
        
        for i in range(k):
            # Split validation set
            val_fold = valset[i * fold_size:(i + 1) * fold_size]
            
            # Optimize pipeline for this fold
            optimized = optimizer.optimize(
                module=pipeline,
                trainset=[],  # Zero-shot learning
                valset=val_fold,
                metric=metric_fn
            )
            
            # Evaluate on remaining data
            score = self._evaluate_pipeline(optimized, valset[:i * fold_size] + valset[(i + 1) * fold_size:])
            
            if score > best_score:
                best_score = score
                best_pipeline = optimized
        
        return best_pipeline

In [ ]:
# Create and optimize pipeline with zero-shot learning
pipeline = ContractPipeline()
optimizer = ZeroShotOptimizer(deployment)
optimized_pipeline = optimizer.optimize(pipeline)

# Test with sample contract
with open(df['filename'].iloc[0], 'r') as f:
    test_contract = f.read()

# Test classification
clause_type = "Non-Disclosure Agreement (NDA) clause"
result = optimized_pipeline(test_contract, clause_type)
print(f"Classification: {result[0]}")
print(f"Reasoning: {result[1]}")